In [34]:
import psycopg2
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [2]:
connection = psycopg2.connect(
    host = 'XXX.XXX.us-east-2.rds.amazonaws.com',
    port = 5432,
    user = 'postgres',
    password = 'XXX',
    database='XXX'
    )
cursor=connection.cursor()

In [3]:
sql = """
SELECT "table_name","column_name", "data_type", "table_schema"
FROM INFORMATION_SCHEMA.COLUMNS
WHERE "table_schema" = 'public'
ORDER BY table_name  
"""
pd.read_sql(sql, con=connection)

,table_name,column_name,data_type,table_schema
0,FPIS,NAME,character varying,public
1,FPIS,FIPS_ID,integer,public
2,TEMPERATURE,DATE,date,public
3,TEMPERATURE,Country,character varying,public
4,TEMPERATURE,AverageTemperatureUncertaintyF,numeric,public
5,TEMPERATURE,STATE,character varying,public
6,TEMPERATURE,TEMPERATURE_ID,integer,public
7,TEMPERATURE,Fahrenheit,numeric,public
8,TEMPERATURE,AverageTemperatureF,numeric,public
9,USDisaster,pa_program_declared,integer,public


In [6]:
sql_USDisaster = """ SELECT * FROM public."USDisaster" ORDER BY declaration_date DESC """
disaster_df = pd.read_sql(sql_USDisaster, con=connection)
disaster_df

,fema_declaration,disaster_number,state,declaration_type,declaration_date,fy_declared,incident_type,declaration_title,ih_program_declared,ia_program_declared,pa_program_declared,hm_program_declared,incident_begin_date,fips,place_code,designated_area,declaration_request_number,hash,last_refresh,id
0,FM-5416-CA,5416,CA,FM,2021-08-25,2021,Fire,French Fire,0,0,1,1,2021-08-24,6029,99029,Kern (County),21088,b2120fdada11e2ceefe3714927afe2443c719812,2021-08-25,61266e5e434e34d5eee6f25e
1,DR-4610-CA,4610,CA,DR,2021-08-24,2021,Fire,Wildfires,1,0,1,1,2021-07-14,6063,99063,Plumas (County),21087,d615785ddbaca2e0dbb66805d84246680c86f560,2021-08-25,6125a937434e34d5ee5cd8af
2,DR-4610-CA,4610,CA,DR,2021-08-24,2021,Fire,Wildfires,1,0,1,1,2021-07-14,6057,99057,Nevada (County),21087,dde89f65556f220034ac16cfa66abf7d135962da,2021-08-25,6125a937434e34d5ee5cd8c1
3,DR-4610-CA,4610,CA,DR,2021-08-24,2021,Fire,Wildfires,1,0,1,1,2021-07-14,6035,99035,Lassen (County),21087,74f77c93f158659bf1dd9f871227a9993244478d,2021-08-25,6125a937434e34d5ee5cd8a3
4,DR-4610-CA,4610,CA,DR,2021-08-24,2021,Fire,Wildfires,1,0,1,1,2021-07-14,6061,99061,Placer (County),21087,6e1ab1c91a9838148684d35413b598c5ea66df71,2021-08-25,6125a937434e34d5ee5cd8b6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61801,DR-5-MT,5,MT,DR,1953-06-06,1953,Flood,Floods,0,1,1,1,1953-06-06,30000,0,Statewide,53006,954449c15634fb45c8bea3ac975782793ccde050,2021-07-13,60c3b7a9a0ee349d71025774
61802,DR-4-MI,4,MI,DR,1953-06-02,1953,Tornado,Tornado,0,1,1,1,1953-06-02,26000,0,Statewide,53004,87a0c1dd5da249767f545e0c0a43f917e4e9ca83,2021-07-13,60c3b7a9a0ee349d7102577a
61803,DR-3-LA,3,LA,DR,1953-05-29,1953,Flood,Flood,0,1,1,1,1953-05-29,22000,0,Statewide,53005,4fb19699fdbba1387ffa2263fcc4a4e37a1de6d6,2021-07-13,60c3b7a9a0ee349d71025777
61804,DR-2-TX,2,TX,DR,1953-05-15,1953,Tornado,Tornado & Heavy Rainfall,0,1,1,1,1953-05-15,48000,0,Statewide,53003,c879557e78d059e6847e7688388c339d10f51979,2021-07-13,60c3b7a9a0ee349d71025783


In [23]:
# drop columns we won't use in model
disaster_df_concise = disaster_df.drop(columns=["declaration_date", "incident_begin_date","fema_declaration", "disaster_number", "declaration_title", "ih_program_declared", "ia_program_declared", "pa_program_declared", "hm_program_declared",  "hash", "last_refresh", "id"])
disaster_df_concise

,state,declaration_type,fy_declared,incident_type,fips,place_code,designated_area,declaration_request_number
0,CA,FM,2021,Fire,6029,99029,Kern (County),21088
1,CA,DR,2021,Fire,6063,99063,Plumas (County),21087
2,CA,DR,2021,Fire,6057,99057,Nevada (County),21087
3,CA,DR,2021,Fire,6035,99035,Lassen (County),21087
4,CA,DR,2021,Fire,6061,99061,Placer (County),21087
...,...,...,...,...,...,...,...,...
61801,MT,DR,1953,Flood,30000,0,Statewide,53006
61802,MI,DR,1953,Tornado,26000,0,Statewide,53004
61803,LA,DR,1953,Flood,22000,0,Statewide,53005
61804,TX,DR,1953,Tornado,48000,0,Statewide,53003


In [24]:
# encode incident type
disaster_encoded = pd.get_dummies(disaster_df_concise, columns=["incident_type"])
disaster_encoded.columns

Index(['state', 'declaration_type', 'fy_declared', 'fips', 'place_code',
       'designated_area', 'declaration_request_number',
       'incident_type_Biological', 'incident_type_Chemical',
       'incident_type_Coastal Storm', 'incident_type_Dam/Levee Break',
       'incident_type_Drought', 'incident_type_Earthquake',
       'incident_type_Fire', 'incident_type_Fishing Losses',
       'incident_type_Flood', 'incident_type_Freezing',
       'incident_type_Human Cause', 'incident_type_Hurricane',
       'incident_type_Mud/Landslide', 'incident_type_Severe Ice Storm',
       'incident_type_Severe Storm(s)', 'incident_type_Snow',
       'incident_type_Terrorist', 'incident_type_Tornado',
       'incident_type_Toxic Substances', 'incident_type_Tsunami',
       'incident_type_Typhoon', 'incident_type_Volcano'],
      dtype='object')

In [25]:
# drop columns so only using hurrican data 
disaster_hurricane = disaster_encoded.drop(columns=['incident_type_Biological',
       'incident_type_Chemical', 'incident_type_Coastal Storm',
       'incident_type_Dam/Levee Break', 'incident_type_Drought',
       'incident_type_Earthquake', 'incident_type_Fire',
       'incident_type_Fishing Losses', 'incident_type_Flood',
       'incident_type_Freezing', 'incident_type_Human Cause','incident_type_Mud/Landslide',
       'incident_type_Severe Ice Storm', 'incident_type_Severe Storm(s)',
       'incident_type_Snow', 'incident_type_Terrorist',
       'incident_type_Tornado', 'incident_type_Toxic Substances',
       'incident_type_Tsunami', 'incident_type_Typhoon',
       'incident_type_Volcano'])
disaster_hurricane.head()

,state,declaration_type,fy_declared,fips,place_code,designated_area,declaration_request_number,incident_type_Hurricane
0,CA,FM,2021,6029,99029,Kern (County),21088,0
1,CA,DR,2021,6063,99063,Plumas (County),21087,0
2,CA,DR,2021,6057,99057,Nevada (County),21087,0
3,CA,DR,2021,6035,99035,Lassen (County),21087,0
4,CA,DR,2021,6061,99061,Placer (County),21087,0


In [26]:
disaster_hurricane.dtypes

state                         object
declaration_type              object
fy_declared                    int64
fips                           int64
place_code                    object
designated_area               object
declaration_request_number     int64
incident_type_Hurricane        uint8
dtype: object

In [27]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
disaster_hurricane["state"] = le.fit_transform(disaster_hurricane["state"])
disaster_hurricane["declaration_type"] = le.fit_transform(disaster_hurricane["declaration_type"])
disaster_hurricane["place_code"] = le.fit_transform(disaster_hurricane["place_code"])
disaster_hurricane["designated_area"] = le.fit_transform(disaster_hurricane["designated_area"])


In [28]:
disaster_hurricane.head()

,state,declaration_type,fy_declared,fips,place_code,designated_area,declaration_request_number,incident_type_Hurricane
0,5,2,2021,6029,1017,1234,21088,0
1,5,0,2021,6063,1040,1872,21087,0
2,5,0,2021,6057,1036,1659,21087,0
3,5,0,2021,6035,1021,1336,21087,0
4,5,0,2021,6061,1039,1864,21087,0


In [29]:
disaster_hurricane.dtypes

state                         int64
declaration_type              int64
fy_declared                   int64
fips                          int64
place_code                    int64
designated_area               int64
declaration_request_number    int64
incident_type_Hurricane       uint8
dtype: object

In [30]:
# split into X and Y valuese
y = disaster_hurricane["incident_type_Hurricane"]
X = disaster_hurricane.drop(columns=["incident_type_Hurricane"])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [32]:
# scale and normalize data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_scaler  = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [35]:
# instantate Logistic Regression
classifier = LogisticRegression(random_state=1)

In [36]:
# fit model
classifier.fit(X_train_scaled, y_train)


LogisticRegression(random_state=1)

In [38]:
#predict
y_predict = classifier.predict(X_test_scaled)

In [40]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_predict)

0.7849469324359306

In [41]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_predict)
cm_df = pd.DataFrame(cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])
cm_df

,Predicted 0,Predicted 1
Actual 0,12040,391
Actual 1,2932,89
